# Montar Drive + Imports

In [1]:
import os
from google.colab import drive
from typing import Dict, Any, List, Union
import pandas as pd
from tqdm import tqdm
import numpy as np
import networkx as nx
import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, f_oneway, ttest_ind, mannwhitneyu, kruskal
from itertools import combinations

In [2]:
# 1. Montar Google Drive
print("Paso 1: Montando Google Drive...")
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Máster - Data Science/TFM/Data/ESTUDIO_MULTIPLEX')

Paso 1: Montando Google Drive...
Mounted at /content/drive


# Función de recuperación de información + listar pacientes

In [3]:
def obtener_datos_paciente(
    patient_id: str,
    base_dir: str = 'DADES_CORRECTED'
) -> Union[Dict[str, Any], None]:
    """
    Recupera toda la información (clínica, nodos y redes) para un paciente
    dado su ID, a partir de la estructura de archivos en base_dir.

    Args:
        patient_id: El ID del paciente (ej. 'P001').
        base_dir: El directorio base donde se encuentran los archivos.

    Returns:
        Un diccionario con la información del paciente en el formato solicitado,
        o None si el ID del paciente no se encuentra.
    """

    # Rutas a los archivos
    clinic_file = os.path.join(base_dir, 'CLINIC.csv')
    nodes_vol_file = os.path.join(base_dir, 'NODES.csv')
    nodes_fa_file = os.path.join(base_dir, 'GM_FA.csv')
    nodes_md_file = os.path.join(base_dir, 'GM_MD.csv')

    # 1. Recuperar información clínica
    try:
        df_clinic = pd.read_csv(clinic_file)

        # Buscar la fila del paciente por su ID
        patient_data = df_clinic[df_clinic['ids'] == patient_id]

        if patient_data.empty:
            print(f"Error: ID de paciente '{patient_id}' no encontrado en CLINIC.csv.")
            return None

        # Extraer los datos (la fila es un DataFrame de 1xN, usamos .iloc[0] para obtener la Serie)
        patient_series = patient_data.iloc[0]

    except FileNotFoundError:
        print(f"Error: Archivo CLINIC.csv no encontrado en {base_dir}")
        return None

    # 2. Mapear y codificar la información clínica al formato solicitado

    # El campo 'origenes' se usa para el campo 'origen' principal
    origen_data = str(patient_series['origenes'])

    # El campo 'sites' se mapea a 'site' en la respuesta
    site_data = str(patient_series['sites'])

    # Asumimos que los campos 'sexes', 'controls_mses' y 'mstypes' están
    # codificados como enteros o pueden ser convertidos a float/int.
    # Necesitarás definir la codificación de 'sexes' y 'mstypes' para el formato final,
    # aquí se usarán los valores tal cual están en el CSV (o se fuerza a int/float).

    # Nota: Los nombres de las columnas ddes y edsses en el CSV anterior eran ddes y edsses
    # pero el formato solicitado pide 'dd' y 'edss'.
    info_clinica = {
        'id': str(patient_series['ids']),
        'age': float(patient_series['ages']),
        # Se asume una codificación para 'sexes', aquí se devuelve como está
        'sex': int(patient_series['sexes']) if pd.notna(patient_series['sexes']) else np.nan,
        'dd': float(patient_series['ddes']),
        'edss': float(patient_series['edsses']),
        'control_ms': int(patient_series['controls_mses']),
        # Se asume una codificación para 'mstypes' (0=RRMS, 1=SPMS, 2=PPMS)
        'mstype': int(patient_series['mstypes']) if pd.notna(patient_series['mstypes']) else np.nan
    }

    # 3. Recuperar datos de nodos (nodos_volumetrico, nodos_FA, nodos_MD)

    def load_node_data(file_path: str) -> List[float]:
        """Carga y devuelve la lista de valores de nodo para el paciente."""
        try:
            df_nodes = pd.read_csv(file_path)
            patient_row = df_nodes[df_nodes['ID'] == patient_id]
            if not patient_row.empty:
                # Excluir la columna 'ID' y convertir los valores a lista (float)
                return patient_row.iloc[0, 1:].tolist()
            else:
                return []
        except FileNotFoundError:
            print(f"Advertencia: Archivo de nodos no encontrado: {file_path}")
            return []

    nodos_volumetrico = load_node_data(nodes_vol_file)
    nodos_FA = load_node_data(nodes_fa_file)
    nodos_MD = load_node_data(nodes_md_file)

    # 4. Recuperar matrices de conectividad (redes)

    redes = {}
    network_types = {'FA': 'FA_network', 'GM': 'GM_network', 'rsfmri': 'rsfmri_network'}

    for net_key, net_dir in network_types.items():
        network_path = os.path.join(base_dir, net_dir, f"{patient_id}.csv")
        try:
            # Leer la matriz sin encabezados ni índice
            matrix = pd.read_csv(network_path, header=None, index_col=None).values
            redes[net_key] = matrix
        except FileNotFoundError:
            print(f"Advertencia: Matriz {net_key} no encontrada para {patient_id} en {network_path}")
            # Si una matriz falta, se puede optar por devolver un array vacío o None
            redes[net_key] = np.array([])

    # 5. Consolidar el resultado

    resultado = {
        'origen': origen_data,
        'site': site_data, # Incluir 'site' ya que está en los datos clínicos
        'info_clinica': info_clinica,
        'nodos_volumetrico': nodos_volumetrico,
        'nodos_FA': nodos_FA,
        'nodos_MD': nodos_MD,
        'redes': redes
    }

    return resultado

In [4]:
def listar_pacientes(base_dir: str = ".") -> list[str]:
    """
    Devuelve una lista con todos los valores de 'id_paciente' del archivo patients.csv.

    Args:
        base_dir (str): Carpeta base donde se encuentra patients.csv

    Returns:
        list[str]: Lista de IDs de pacientes.
    """
    patients_path = os.path.join(base_dir, "patients.csv")

    if not os.path.exists(patients_path):
        raise FileNotFoundError(f"No se encontró el archivo: {patients_path}")

    df_patients = pd.read_csv(patients_path)

    if "id_paciente" not in df_patients.columns:
        raise ValueError("El archivo patients.csv no contiene la columna 'id_paciente'.")

    # Elimina valores nulos y los convierte a string
    ids = df_patients["id_paciente"].dropna().astype(str).tolist()

    return ids

pacientes = listar_pacientes()

# Cargar los datos

Obtenemos los datos en formato tensorial para procesarlos luego:

- `FA_matrices_tensor[i][j]`, `GM_matrices_tensor[i][j]` y `rsfMRI_matrices_tensor[i][j]` → lista de longitud `n_pacientes` con todos los valores de la conexión `(i,j)` para todos los pacientes.

- `nodos_volumetricos_tensor[k]`, `nodos_FA_tensor[k]` y `nodos_MD_tensor[k]` → listas de valores por nodo `k` para todos los pacientes.

- `origen`, `ages`, `sexes`, `ddes`, `edsses`, `control_mses`, `mstypes`  → arrays con las variables clínicas.

- `site`  → array con el protocolo de muestra (FIS, MSVIS o sub)

In [5]:
# --- Obtener dimensiones base (suponiendo todos los pacientes tienen mismas dimensiones) ---
info_ref = obtener_datos_paciente(pacientes[0])
n_nodos = info_ref['redes']['FA'].shape[0]

# --- Inicializar estructuras vacías ---
FA_matrices_tensor = [[[] for _ in range(n_nodos)] for _ in range(n_nodos)]
GM_matrices_tensor = [[[] for _ in range(n_nodos)] for _ in range(n_nodos)]
rsfMRI_matrices_tensor = [[[] for _ in range(n_nodos)] for _ in range(n_nodos)]

nodos_volumetricos_tensor = [[] for _ in range(n_nodos)]
nodos_FA_tensor = [[] for _ in range(n_nodos)]
nodos_MD_tensor = [[] for _ in range(n_nodos)]

# Variables clínicas
origenes = []
ids = []
sites = []
ages = []
sexes = []
ddes = []
edsses = []
controls_mses = []
mstypes = []

# --- Iterar sobre pacientes ---
for paciente in tqdm(pacientes, desc="Construyendo tensores de conectividad"):
    info = obtener_datos_paciente(paciente)
    redes = info['redes']
    clinica = info['info_clinica']

    # Guardar variables clínicas
    origenes.append(info['origen'])
    ids.append(clinica['id'])
    sites.append(info['site'])
    ages.append(clinica['age'])
    sexes.append(clinica['sex'])
    controls_mses.append(clinica['control_ms'])
    ddes.append(clinica['dd'])
    edsses.append(clinica['edss'])
    mstypes.append(clinica['mstype'])

    # --- Rellenar tensores de conectividad ---
    for i in range(n_nodos):
        for j in range(n_nodos):
            FA_matrices_tensor[i][j].append(redes['FA'][i, j])
            GM_matrices_tensor[i][j].append(redes['GM'][i, j])
            rsfMRI_matrices_tensor[i][j].append(redes['rsfmri'][i, j])

    # --- Rellenar tensores de nodos ---
    for k in range(n_nodos):
        nodos_volumetricos_tensor[k].append(info['nodos_volumetrico'][k])
        nodos_FA_tensor[k].append(info['nodos_FA'][k])
        nodos_MD_tensor[k].append(info['nodos_MD'][k])


# Convertir a numpy
FA_matrices_tensor = np.array(FA_matrices_tensor) # Dimensión: (n_nodos, n_nodos, n_pacientes)
GM_matrices_tensor = np.array(GM_matrices_tensor)
rsfMRI_matrices_tensor = np.array(rsfMRI_matrices_tensor)

nodos_volumetricos_tensor = np.array(nodos_volumetricos_tensor)
nodos_FA_tensor = np.array(nodos_FA_tensor)
nodos_MD_tensor = np.array(nodos_MD_tensor)


# Convertir edad y sexo a arrays NumPy
ages = np.array(ages)
sexes = np.array(sexes)
ddes = np.array(ddes)
edsses = np.array(edsses)
controls_mses = np.array(controls_mses)
mstypes = np.array(mstypes)

Construyendo tensores de conectividad: 100%|██████████| 270/270 [09:40<00:00,  2.15s/it]


# Cálculos 1-layer (global)

In [ ]:
def calcular_metricas_grafo(matriz_conectividad):
    """
    Calcula diversas métricas de un grafo a partir de su matriz de conectividad.

    :param matriz_conectividad: numpy.ndarray, matriz de conectividad/pesos
                                donde nan indica ausencia de conexión.
                                Se asume un grafo no dirigido.
    :return: dict, diccionario con las métricas calculadas.
    """
    if not isinstance(matriz_conectividad, np.ndarray):
        matriz_conectividad = np.array(matriz_conectividad)

    # 1. Preprocesamiento: Convertir la matriz de conectividad a una
    #    matriz de adyacencia/pesos válida para NetworkX.
    #    - Los nan se reemplazan por 0, indicando "sin arista".
    #    - Se asume que los valores no nan son los pesos de las aristas.

    # Crear una copia para no modificar la original
    adj_matrix = matriz_conectividad.copy()

    # Reemplazar nan por 0. Esto crea la matriz de adyacencia (A[i,j] = 0 si no hay arista)
    # y maneja correctamente si el peso es 0 (que sí es una arista con peso 0).
    adj_matrix[np.isnan(adj_matrix)] = 0

    # Forzar la matriz a ser simétrica para asumir un grafo no dirigido
    # (aunque esto puede cambiar si el grafo es dirigido)
    adj_matrix = np.maximum(adj_matrix, adj_matrix.T)

    # Grafo basado en los pesos
    # 1. Aplicar la lógica: Si un valor es > 1, se convierte en 1.
    adj_matrix_fuerza = adj_matrix.copy()
    G_fuerza = nx.from_numpy_array(adj_matrix_fuerza, create_using=nx.Graph)

    # Convertir pesos distintos de 0 en distancias inversas (1/w)
    adj_matrix[adj_matrix != 0] = 1.0 / adj_matrix[adj_matrix != 0]

    # 2. Creación del objeto Grafo
    # Se crea un grafo ponderado (Weighted Graph) a partir de la matriz de adyacencia.
    # Los aristas con peso 0 (o nan original) no serán incluidas.
    G = nx.from_numpy_array(adj_matrix, create_using=nx.Graph)

    # Eliminar aristas con peso 0 (si se deben considerar como "no conexión")
    # NetworkX crea aristas para A[i,j] > 0. Si A[i,j] = 0 (por el nan o por el valor original),
    # no se crea la arista, lo que es el comportamiento deseado.

    # 3. Preparar para métricas que requieren que el grafo esté conectado
    #    (Longitud de Camino Característico, Eficiencia Global)

    # Obtener el subgrafo conectado más grande (Large Connected Component - LCC)
    if nx.is_connected(G):
        LCC = G
    else:
        # Encontrar el subgrafo con más nodos
        componentes = list(nx.connected_components(G))
        if not componentes:
             # Grafo vacío o sin aristas
            warnings.warn("El grafo está vacío o no tiene aristas.")
            return {
                "Eficiencia global": np.nan,
                "Longitud de camino característico": np.nan,
                "Fuerza nodal promedio": 0.0,
                "Coeficiente de agrupamiento": 0.0,
                "Asortatividad": np.nan,
                "Transitividad": 0.0,
                "Modularidad": np.nan
            }

        nodos_lcc = max(componentes, key=len)
        LCC = G.subgraph(nodos_lcc).copy()
        warnings.warn("El grafo no está conectado. Las métricas de camino/distancia se calculan en la Componente Conectada más Grande (LCC).")

    N = G.number_of_nodes()
    N_lcc = LCC.number_of_nodes()

    # 4. Cálculo de Métricas
    metricas = {}

    # --- Métricas Globales de Distancia (requieren LCC) ---
    # NetworkX calcula las distancias usando los pesos ('weight') por defecto

    # **Longitud de Camino Característico (Average Shortest Path Length)**
    # Mide el grado promedio de separación.
    if N_lcc > 1:
        # Se calcula en la LCC, ya que un grafo desconectado tiene una longitud infinita
        # NetworkX usa el peso para calcular la distancia.
        try:
             metricas["Longitud de camino característico"] = nx.average_shortest_path_length(LCC, weight='weight')
        except nx.NetworkXNoPath:
             metricas["Longitud de camino característico"] = np.nan # No hay camino entre algunos pares
    else:
        metricas["Longitud de camino característico"] = 0.0

    # **Eficiencia Global (Global Efficiency)**
    def global_efficiency_weighted(G, weight='weight'):
        nodes = list(G.nodes())
        N = len(nodes)
        if N <= 1:
            return 0.0

        # Obtener todas las distancias ponderadas
        dist = dict(nx.all_pairs_dijkstra_path_length(G, weight=weight))

        suma = 0
        for i in nodes:
            for j in nodes:
                if i != j:
                    d = dist[i].get(j, np.inf)
                    if d != np.inf:
                        suma += 1 / d

        return suma / (N * (N - 1))

    # E_global = 1/|V| * sum_{i!=j} (1/d_ij). Mide la eficiencia de comunicación.
    if N > 1:
        # Se puede calcular en el grafo completo (G), ignorando caminos infinitos (1/inf=0)
        # NetworkX calcula la eficiencia global, maneja desconexiones internamente.
        metricas["Eficiencia global"] = global_efficiency_weighted(G, weight='weight')
    else:
        metricas["Eficiencia global"] = 0.0

    # --- Métricas de Grado y Fuerza ---

    # **Fuerza Nodal Promedio (Average Node Strength)**
    # La fuerza de un nodo es la suma de los pesos de sus aristas (análogo al grado en grafos no ponderados).
    # Se calcula la suma de los pesos de las aristas (fuerza/strength) para cada nodo y luego se promedia.
    node_strengths = G_fuerza.degree(weight='weight')
    total_strength = sum(s for _, s in node_strengths)
    metricas["Fuerza nodal promedio"] = total_strength / N if N > 0 else 0.0

    # --- Métricas de Agrupamiento ---

    # **Coeficiente de Agrupamiento (Clustering Coefficient)**
    metricas["Coeficiente de agrupamiento"] = nx.average_clustering(G_fuerza, weight='weight')

    # **Transitividad (Transitivity)**
    # Mide la probabilidad de que un triángulo exista en el grafo (Global Clustering).
    metricas["Transitividad"] = nx.transitivity(G_fuerza)

    # --- Métricas de Conectividad y Estructura ---

    # **Asortatividad (Assortativity)**
    # Mide si los nodos se conectan preferentemente con nodos de grado similar.
    # En grafos ponderados, se usa la asortatividad de grado (ignorando pesos)
    try:
        metricas["Asortatividad"] = nx.degree_assortativity_coefficient(G_fuerza, weight="weight")
    except Exception:
        metricas["Asortatividad"] = np.nan # Puede fallar en grafos muy pequeños o vacíos

    # **Modularidad (Modularity)**
    # Mide la fuerza de la división de un grafo en módulos (comunidades).
    # Requiere encontrar las comunidades primero. Usamos el algoritmo de Louvain.
    try:
        # El algoritmo de Louvain busca la partición que maximiza la modularidad.
        # Usa el peso si está presente.
        comunidades = nx.community.louvain_communities(G_fuerza, weight='weight')
        # La modularidad se calcula en base a la partición encontrada
        metricas["Modularidad"] = nx.community.modularity(G_fuerza, comunidades, weight='weight')
    except Exception as e:
        # Esto puede fallar si el grafo es trivial o muy disperso.
        metricas["Modularidad"] = np.nan
        warnings.warn(f"No se pudo calcular la Modularidad: {e}")

    # 5. Formato de Salida
    # Asegurar que todos los valores sean flotantes o NaN para consistencia
    for k, v in metricas.items():
        if isinstance(v, (int, np.integer)):
            metricas[k] = float(v)
        elif isinstance(v, (float, np.floating)):
            pass # Ya es un float/numpy float
        elif v is None:
            metricas[k] = np.nan

    return metricas

In [10]:
def calcular_metricas_tensor(tensor, ids, nombre_salida, calcular_metricas_grafo):
    """
    Itera por la tercera dimensión de un tensor (i, j, k),
    calcula métricas de grafo para cada matriz k y
    guarda los resultados en un CSV en formato:

    id, metrica1, metrica2, ..., metrican
    """

    carpeta = "METRICAS_GRAFO"
    os.makedirs(carpeta, exist_ok=True)

    resultados_fila = []

    # tqdm envuelve la iteración para mostrar la barra de progreso
    for k in tqdm(range(tensor.shape[2]), desc="Calculando métricas"):
        matriz = tensor[:, :, k]
        metricas = calcular_metricas_grafo(matriz)   # Diccionario

        fila = {"id": ids[k]}
        fila.update(metricas)   # Añade cada métrica como columna

        resultados_fila.append(fila)

    df = pd.DataFrame(resultados_fila)

    ruta_salida = os.path.join(carpeta, f"{nombre_salida}.csv")
    df.to_csv(ruta_salida, index=False)

    print(f"CSV guardado en: {ruta_salida}")

In [ ]:
calcular_metricas_tensor(GM_matrices_tensor, ids, "GM_GLOBAL", calcular_metricas_grafo)
calcular_metricas_tensor(FA_matrices_tensor, ids, "FA_GLOBAL", calcular_metricas_grafo)
calcular_metricas_tensor(rsfMRI_matrices_tensor, ids, "rsfMRI_GLOBAL", calcular_metricas_grafo)

Calculando métricas:   0%|          | 0/270 [00:00<?, ?it/s]/tmp/ipython-input-1214741562.py:70: UserWarning: El grafo no está conectado. Las métricas de camino/distancia se calculan en la Componente Conectada más Grande (LCC).
  warnings.warn("El grafo no está conectado. Las métricas de camino/distancia se calculan en la Componente Conectada más Grande (LCC).")
Calculando métricas: 100%|██████████| 270/270 [02:41<00:00,  1.68it/s]


CSV guardado en: METRICAS_GRAFO/GM_GLOBAL.csv


Calculando métricas: 100%|██████████| 270/270 [01:52<00:00,  2.41it/s]


CSV guardado en: METRICAS_GRAFO/FA_GLOBAL.csv


Calculando métricas: 100%|██████████| 270/270 [02:28<00:00,  1.82it/s]

CSV guardado en: METRICAS_GRAFO/rsfMRI_GLOBAL.csv


# Cálculos 1-layer (local)

In [ ]:
def calcular_metricas_grafo_local(matriz_conectividad):
    """
    Calcula diversas métricas de un grafo a partir de su matriz de conectividad.

    Métricas calculadas a nivel nodal:
        - Fuerza Nodal (suma de pesos de aristas)
        - Grado Nodal (número de aristas)
        - Eficiencia Local
        - Centralidad de intermediación (betweenness)
        - Centralidad de cercanía (closeness)

    :param matriz_conectividad: numpy.ndarray, matriz de conectividad/pesos
                                donde nan indica ausencia de conexión.
                                Se asume un grafo no dirigido.
    :return: dict, diccionario con las métricas calculadas (arrays de n nodos por métrica).
    """
    if not isinstance(matriz_conectividad, np.ndarray):
        matriz_conectividad = np.array(matriz_conectividad)

    # --- Preprocesamiento de Fuerza ---
    # 1. Copia y reemplazo de NaN por 0
    adj_fuerza = matriz_conectividad.copy()
    adj_fuerza[np.isnan(adj_fuerza)] = 0

    # 2. Simetrización para grafo no dirigido
    adj_fuerza = np.maximum(adj_fuerza, adj_fuerza.T)

    # Crear grafo con la FUERZA de conexión como 'weight'
    G_fuerza = nx.from_numpy_array(adj_fuerza, create_using=nx.Graph)

    # --- Preprocesamiento de Distancia (para métricas de camino) ---
    adj_distancia = adj_fuerza.copy()
    # Invertir pesos: Distancia = 1 / Fuerza
    with np.errstate(divide='ignore'): # Ignorar división por cero (aristas con fuerza 0)
        adj_distancia[adj_distancia != 0] = 1.0 / adj_distancia[adj_distancia != 0]

    # Crear grafo con la DISTANCIA como 'weight'
    G_distancia = nx.from_numpy_array(adj_distancia, create_using=nx.Graph)

    # --- Cálculo de métricas ---

    # Fuerza Nodal: suma de las FUERZAS de las aristas (usando G_fuerza)
    fuerza_nodal = np.array([d for n, d in G_fuerza.degree(weight='weight')])

    # Grado Nodal: número de aristas conectadas a cada nodo (usando G_fuerza o G_distancia)
    grado_nodal = np.array([d for n, d in G_fuerza.degree()])

    def calcular_eficiencia_local_nodal(G):
        """
        Calcula la eficiencia local (E_i) para cada nodo i en el grafo G.
        E_i = eficiencia global del subgrafo inducido por los vecinos de i.
        Se calcula una sola vez todas las distancias y luego se usa para cada subgrafo.
        """

        nodes = list(G.nodes())
        N = len(nodes)

        # 1. Calcular todas las distancias ponderadas una sola vez
        # Distancias entre todos los pares de nodos (peso = 'weight')
        dist_all = dict(nx.all_pairs_dijkstra_path_length(G, weight='weight'))

        eficiencia_local_por_nodo = {}

        for nodo_i in nodes:
            vecinos_i = list(G.neighbors(nodo_i))

            if len(vecinos_i) < 2:
                eficiencia_local_por_nodo[nodo_i] = 0.0
                continue

            # 2. Calcular eficiencia global del subgrafo inducido por los vecinos usando distancias precalculadas
            suma = 0.0
            for u in vecinos_i:
                for v in vecinos_i:
                    if u != v:
                        d = dist_all[u].get(v, np.inf)
                        if d != np.inf:
                            suma += 1 / d

            k = len(vecinos_i)
            eficiencia_local_por_nodo[nodo_i] = suma / (k * (k - 1))

        # Aseguramos el orden del array para que coincida con el orden de los nodos de NetworkX
        return np.array([eficiencia_local_por_nodo[n] for n in nodes])

    eficiencia_local = calcular_eficiencia_local_nodal(G_distancia)


    # Centralidad de intermediación (betweenness)
    # distance='weight' usa los pesos del grafo (distancias)
    betweenness = np.array(list(nx.betweenness_centrality(G_distancia, weight='weight').values()))

    # Centralidad de cercanía (closeness)
    # distance='weight' usa los pesos del grafo (distancias)
    closeness = np.array(list(nx.closeness_centrality(G_distancia, distance='weight').values()))

    # --- Retornar resultados ---
    return {
        'fuerza_nodal': fuerza_nodal,
        'grado_nodal': grado_nodal,
        'eficiencia_local': eficiencia_local,
        'centralidad_intermediacion': betweenness,
        'centralidad_cercania': closeness
    }

In [ ]:
def calcular_metricas_tensor_local_por_nodo(tensor, ids, nombre_salida, calcular_metricas_grafo_local):
    """
    Itera por la tercera dimensión de un tensor (i, j, k),
    calcula métricas locales de grafo para cada matriz k y
    guarda los resultados en CSVs separados para cada métrica local.

    El CSV tiene el formato:
        ID, nodo1, nodo2, ..., nodon
    """
    carpeta = "METRICAS_GRAFO"
    os.makedirs(carpeta, exist_ok=True)

    # Inicializamos diccionarios para cada métrica
    metricas_dict = {}

    for k in tqdm(range(tensor.shape[2]), desc="Procesando matrices"):
        matriz = tensor[:, :, k]
        metricas = calcular_metricas_grafo_local(matriz)  # Diccionario de arrays (nodos)

        for nombre, valores in metricas.items():
            if nombre not in metricas_dict:
                metricas_dict[nombre] = []
            # Añadimos una fila: [ID, val_nodo1, val_nodo2, ...]
            metricas_dict[nombre].append([ids[k]] + list(valores))

    # Guardar cada métrica en un CSV separado
    for nombre, filas in metricas_dict.items():
        df = pd.DataFrame(filas)
        # Nombres de columnas: ID + nodo1, nodo2, ...
        n_nodos = tensor.shape[0]
        columnas = ['ID'] + [f'nodo{i+1}' for i in range(n_nodos)]
        df.columns = columnas

        ruta_salida = os.path.join(carpeta, f"{nombre}_{nombre_salida}.csv")
        df.to_csv(ruta_salida, index=False)
        print(f"CSV guardado en: {ruta_salida}")

In [ ]:
calcular_metricas_tensor_local_por_nodo(GM_matrices_tensor, ids, "GM_LOCAL", calcular_metricas_grafo_local)
calcular_metricas_tensor_local_por_nodo(FA_matrices_tensor, ids, "FA_LOCAL", calcular_metricas_grafo_local)
calcular_metricas_tensor_local_por_nodo(rsfMRI_matrices_tensor, ids, "rsfMRI_LOCAL", calcular_metricas_grafo_local)

Procesando matrices: 100%|██████████| 270/270 [02:20<00:00,  1.92it/s]


CSV guardado en: METRICAS_GRAFO/fuerza_nodal_GM_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/grado_nodal_GM_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/eficiencia_local_GM_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/centralidad_intermediacion_GM_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/centralidad_cercania_GM_LOCAL.csv


Procesando matrices: 100%|██████████| 270/270 [01:41<00:00,  2.66it/s]


CSV guardado en: METRICAS_GRAFO/fuerza_nodal_FA_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/grado_nodal_FA_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/eficiencia_local_FA_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/centralidad_intermediacion_FA_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/centralidad_cercania_FA_LOCAL.csv


Procesando matrices: 100%|██████████| 270/270 [02:07<00:00,  2.11it/s]


CSV guardado en: METRICAS_GRAFO/fuerza_nodal_rsfMRI_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/grado_nodal_rsfMRI_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/eficiencia_local_rsfMRI_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/centralidad_intermediacion_rsfMRI_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/centralidad_cercania_rsfMRI_LOCAL.csv


# Cálculos Multi-Layer (global)

In [11]:
def combinar_tensores(t1, t2, t3):
    """
    t1, t2, t3: tensores de tamaño (n_nodos, n_nodos, n_pacientes)

    Devuelve:
        tensor_out de tamaño (2*n_nodos, 2*n_nodos, n_pacientes)
        tal que para cada paciente p:

            ( t1[:,:,p]   t2[:,:,p] )
            ( t2[:,:,p]   t3[:,:,p] )
    """

    n_nodos, _, n_pacientes = t1.shape

    # Tensor de salida
    out = np.zeros((2*n_nodos, 2*n_nodos, n_pacientes))

    # Rellenar bloques
    out[:n_nodos, :n_nodos, :] = t1
    out[:n_nodos, n_nodos:, :] = t2
    out[n_nodos:, :n_nodos, :] = t2
    out[n_nodos:, n_nodos:, :] = t3

    return out

In [12]:
multilayer_matrices_tensor = combinar_tensores(GM_matrices_tensor, FA_matrices_tensor, rsfMRI_matrices_tensor)

In [13]:
def calcular_metricas_multigrafo(matriz_conectividad):
    """
    Calcula diversas métricas de un grafo a partir de su matriz de conectividad.

    :param matriz_conectividad: numpy.ndarray, matriz de conectividad/pesos
                                donde nan indica ausencia de conexión.
                                Se asume un grafo no dirigido.
    :return: dict, diccionario con las métricas calculadas.
    """
    if not isinstance(matriz_conectividad, np.ndarray):
        matriz_conectividad = np.array(matriz_conectividad)

    # 1. Preprocesamiento: Convertir la matriz de conectividad a una
    #    matriz de adyacencia/pesos válida para NetworkX.
    #    - Los nan se reemplazan por 0, indicando "sin arista".
    #    - Se asume que los valores no nan son los pesos de las aristas.

    # Crear una copia para no modificar la original
    adj_matrix = matriz_conectividad.copy()

    # Reemplazar nan por 0. Esto crea la matriz de adyacencia (A[i,j] = 0 si no hay arista)
    # y maneja correctamente si el peso es 0 (que sí es una arista con peso 0).
    adj_matrix[np.isnan(adj_matrix)] = 0

    # Forzar la matriz a ser simétrica para asumir un grafo no dirigido
    # (aunque esto puede cambiar si el grafo es dirigido)
    adj_matrix = np.maximum(adj_matrix, adj_matrix.T)

    # Grafo basado en los pesos
    adj_matrix_fuerza = adj_matrix.copy()
    G_fuerza = nx.from_numpy_array(adj_matrix_fuerza, create_using=nx.Graph)

    # Convertir pesos distintos de 0 en distancias inversas (1/w)
    adj_matrix[adj_matrix != 0] = 1.0 / adj_matrix[adj_matrix != 0]

    # 2. Creación del objeto Grafo
    # Se crea un grafo ponderado (Weighted Graph) a partir de la matriz de adyacencia.
    # Los aristas con peso 0 (o nan original) no serán incluidas.
    G = nx.from_numpy_array(adj_matrix, create_using=nx.Graph)

    # Eliminar aristas con peso 0 (si se deben considerar como "no conexión")
    # NetworkX crea aristas para A[i,j] > 0. Si A[i,j] = 0 (por el nan o por el valor original),
    # no se crea la arista, lo que es el comportamiento deseado.

    # 3. Preparar para métricas que requieren que el grafo esté conectado
    #    (Longitud de Camino Característico, Eficiencia Global)

    # Obtener el subgrafo conectado más grande (Large Connected Component - LCC)
    if nx.is_connected(G):
        LCC = G
    else:
        # Encontrar el subgrafo con más nodos
        componentes = list(nx.connected_components(G))
        if not componentes:
             # Grafo vacío o sin aristas
            warnings.warn("El grafo está vacío o no tiene aristas.")
            return {
                "Eficiencia global": np.nan,
                "Longitud de camino característico": np.nan,
                "Fuerza nodal promedio": 0.0,
                "Coeficiente de agrupamiento": 0.0,
                "Asortatividad": np.nan,
                "Transitividad": 0.0,
                "Modularidad": np.nan
            }

        nodos_lcc = max(componentes, key=len)
        LCC = G.subgraph(nodos_lcc).copy()
        warnings.warn("El grafo no está conectado. Las métricas de camino/distancia se calculan en la Componente Conectada más Grande (LCC).")

    N = G.number_of_nodes()
    N_lcc = LCC.number_of_nodes()

    # 4. Cálculo de Métricas
    metricas = {}

    # --- Métricas Globales de Distancia (requieren LCC) ---
    # NetworkX calcula las distancias usando los pesos ('weight') por defecto

    # **Longitud de Camino Característico (Average Shortest Path Length)**
    # Mide el grado promedio de separación.
    n_nodos = LCC.number_of_nodes()//2
    # 1️⃣ Calcular todas las distancias usando Dijkstra
    dist_dict = dict(nx.all_pairs_dijkstra_path_length(LCC, weight='weight'))

    # 2️⃣ Inicializar la matriz de distancias completas
    dist_matrix = np.full((N, N), np.inf)
    nodes = list(LCC.nodes())
    node_index = {node: idx for idx, node in enumerate(nodes)}

    for u, lengths in dist_dict.items():
        i = node_index[u]
        for v, d in lengths.items():
            j = node_index[v]
            dist_matrix[i, j] = d

    # 3️⃣ Construir la matriz mínima considerando el nodo duplicado
    # En verdad es innecesario ya que si se pone que la distancia de
    # nodo a nodo (i a i) entre capas es 0 entonces es lo mismo
    dist_matrix_min = np.minimum(
        dist_matrix[:n_nodos, :n_nodos],      # original
        dist_matrix[:n_nodos, n_nodos:2*n_nodos]  # duplicada
    )

    # 4️⃣ Calcular la longitud promedio del camino más corto
    finite_distances = dist_matrix_min[np.isfinite(dist_matrix_min)]
    if len(finite_distances) > 0:
        metricas["Longitud de camino característico"] = np.mean(finite_distances)
    else:
        metricas["Longitud de camino característico"] = np.nan

    # **Eficiencia Global (Global Efficiency)**
    def global_efficiency_weighted(dist_matrix_min, n_nodos):
        # Calcular eficiencia global
        suma = 0
        for i in range(n_nodos):
            for j in range(n_nodos):
                if i != j and np.isfinite(dist_matrix_min[i, j]):
                    suma += 1 / dist_matrix_min[i, j]

        return suma / (n_nodos * (n_nodos - 1))

    # E_global = 1/|V| * sum_{i!=j} (1/d_ij). Mide la eficiencia de comunicación.
    if N > 1:
        # Se puede calcular en el grafo completo (G), ignorando caminos infinitos (1/inf=0)
        # NetworkX calcula la eficiencia global, maneja desconexiones internamente.
        metricas["Eficiencia global"] = global_efficiency_weighted(dist_matrix_min, n_nodos)
    else:
        metricas["Eficiencia global"] = 0.0

    # --- Métricas de Fuerza ---

    # **Fuerza Nodal Promedio (Average Node Strength)**
    # La fuerza de un nodo es la suma de los pesos de sus aristas (análogo al grado en grafos no ponderados).
    # Se calcula la suma de los pesos de las aristas (fuerza/strength) para cada nodo y luego se promedia.
    node_strengths = G_fuerza.degree(weight='weight')
    total_strength = sum(s for _, s in node_strengths)
    metricas["Fuerza nodal promedio"] = total_strength / N if N > 0 else 0.0


    # --- Métricas de Agrupamiento ---

    # **Coeficiente de Agrupamiento (Clustering Coefficient)**
    # Basado en Onnela (2005) pero el número posible de enlaces es k_i(k_i-1)/2
    # y también los interlayer que son k_i*k_j si es unilayer y w=1 es cálculo
    # clásico

    def multilayer_clustering_transitivity(G, n_nodos, weight='weight'):
        """
        Calcula el clustering y la transitividad de un grafo multilayer ponderado.

        Parámetros:
        - G (nx.Graph): Grafo multicapa. Nodos esperados: (nodo_base, capa).
        - n_nodos (int): Número de nodos base.
        - weight (str): Nombre del atributo de peso de los enlaces. Por defecto 'weight'.

        Retorna:
        - metricas (dict): Diccionario con el clustering promedio y la transitividad.
        """

        clustering_values = []
        total_triangles = 0
        total_triplets = 0


        # Recorremos cada nodo base
        for i in range(n_nodos):
            # Nodo en la capa 0 e nodo en la capa 1
            node0 = i
            node1 = i + n_nodos

            neighbors0 = list(G.neighbors(node0))
            neighbors1 = list(G.neighbors(node1))

            # Función auxiliar para calcular el clustering ponderado de un nodo dado
            def weighted_clustering_for_node(node, neighbors):
                triangles = 0.0
                ti = 0
                for u, v in combinations(neighbors, 2):
                    if G.has_edge(u, v):
                        w_uv = G[u][v].get(weight, 1.0)
                        w_nu = G[node][u].get(weight, 1.0)
                        w_nv = G[node][v].get(weight, 1.0)
                        triangles += (w_nu * w_nv * w_uv) ** (1/3)
                        ti += 1
                if len(neighbors) < 2:
                    return 0
                return triangles, ti

            # Clustering intralayer
            c0, t0 = weighted_clustering_for_node(node0, neighbors0)
            c1, t1 = weighted_clustering_for_node(node1, neighbors1)

            # Clustering interlayer: vecinos de node0 con vecinos de node1
            triangles_inter = 0.0
            t_inter = 0
            for u in neighbors0:
                for v in neighbors1:
                    if G.has_edge(u, v):
                        w_uv = G[u][v].get(weight, 1.0)
                        w_nu = G[node0][u].get(weight, 1.0)
                        w_nv = G[node1][v].get(weight, 1.0)
                        triangles_inter += (w_nu * w_nv * w_uv) ** (1/3)
                        t_inter += 1
            c_inter = triangles_inter if neighbors0 and neighbors1 else 0

            # Promediamos los tres componentes
            clustering_values.append((c0 + c1 + c_inter)/(len(neighbors0)*len(neighbors1) + len(neighbors0)*(len(neighbors0)-1)/2 + len(neighbors1)*(len(neighbors1)-1)/2))

            # --- Transitividad no ponderada ---
            # Triángulos intralayer
            t0 = sum(1 for u, v in combinations(neighbors0, 2) if G.has_edge(u, v))
            t1 = sum(1 for u, v in combinations(neighbors1, 2) if G.has_edge(u, v))
            trip0 = len(neighbors0) * (len(neighbors0) - 1) / 2
            trip1 = len(neighbors1) * (len(neighbors1) - 1) / 2

            total_triangles += t0 + t1 + t_inter
            total_triplets += (len(neighbors0)*len(neighbors1) + len(neighbors0)*(len(neighbors0)-1)/2 + len(neighbors1)*(len(neighbors1)-1)/2)

        avg_clustering = sum(clustering_values)/len(clustering_values) if clustering_values else 0
        transitivity_global = (3 * total_triangles / total_triplets) if total_triplets > 0 else 0

        return avg_clustering, transitivity_global

    # **Transitividad (Transitivity)**
    # Mide la probabilidad de que un triángulo exista en el grafo (Global Clustering).

    metricas["Coeficiente de agrupamiento"], metricas["Transitividad"] = multilayer_clustering_transitivity(G_fuerza, n_nodos, weight='weight')

    # --- Métricas de Conectividad y Estructura ---

    # **Asortatividad (Assortativity)**
    def manual_degree_assortativity(G, weight=None):
        """
        Calcula la asortatividad por grado (ponderada o no)
        sin usar las funciones de assortativity de NetworkX.
        """

        # --- Obtener la lista de grados ---
        degrees_init = dict(G.degree())

        # Calcular el degree a nivel nodal como suma total

        degrees = {}

        for i in range(n_nodos):
            valor = degrees_init[i] + degrees_init[i + n_nodos]
            degrees[i] = valor
            degrees[i + n_nodos] = valor

        # --- Preparar sumatorias ---
        sum_w = 0.0              # suma de pesos totales (o número de aristas si no hay peso)
        sum_jk = 0.0             # sumatoria de j_i k_i w_i
        sum_j2k2 = 0.0           # sumatoria de (j_i^2 + k_i^2)/2 * w_i
        sum_jk_avg2 = 0.0        # sumatoria de ((j_i + k_i)^2)/2 * w_i

        # --- Recorrer aristas ---
        for u, v, data in G.edges(data=True):

            # obtener peso si corresponde
            w = data.get(weight, 1.0) if weight is not None else 1.0

            j = degrees[u]
            k = degrees[v]

            sum_w += w
            sum_jk += w * j * k
            sum_j2k2 += w * (j**2 + k**2) / 2
            sum_jk_avg2 += w * (j + k)**2 / 2

        # --- Fórmula de Newman (NetworkX la implementa así) ---
        num = sum_jk - (sum_jk_avg2 / sum_w)
        den = sum_j2k2 - (sum_jk_avg2 / sum_w)

        if den == 0:
            return 0  # o NaN, como prefieras

        return num / den

    # Mide si los nodos se conectan preferentemente con nodos de grado similar.
    # En grafos ponderados, se usa la asortatividad de grado (ignorando pesos)
    try:
        metricas["Asortatividad"] = manual_degree_assortativity(G_fuerza, weight="weight")
    except Exception:
        metricas["Asortatividad"] = np.nan # Puede fallar en grafos muy pequeños o vacíos

    # **Modularidad (Modularity)**
    # Proyectamos sobre un grafo unilayer que tenga las conexiones múltiples
    # como el promedio de ellas
    def proyectar_grafo(G_fuerza, n_nodos):
        G_proy = nx.Graph()

        # 1. Crear el grafo proyectado con todos los nodos
        for i in range(n_nodos):
            G_proy.add_node(i)

        # 2. Recorrer todos los pares de nodos base (u, v)
        for u in range(n_nodos):
            for v in range(u + 1, n_nodos):
                # Acumulador de fuerza
                fuerza = 0.0

                # Contribución de la Capa 0 (u -> v)
                if G_fuerza.has_edge(u, v):
                    fuerza += G_fuerza[u][v].get('weight', 0)

                # Contribución de la Capa 1 (u+n -> v+n)
                if G_fuerza.has_edge(u + n_nodos, v + n_nodos):
                    fuerza += G_fuerza[u + n_nodos][v + n_nodos].get('weight', 0)

                # Contribución Inter-Capa (u -> v+n y v -> u+n)
                if G_fuerza.has_edge(u, v + n_nodos):
                    fuerza += G_fuerza[u][v + n_nodos].get('weight', 0)
                if G_fuerza.has_edge(v, u + n_nodos):
                    fuerza += G_fuerza[v][u + n_nodos].get('weight', 0)

                # 3. Añadir la arista al grafo proyectado si hay alguna fuerza
                if fuerza > 0:
                    G_proy.add_edge(u, v, weight=fuerza)

        return G_proy

    G_fuerza_proy = proyectar_grafo(G_fuerza, n_nodos)


    # Mide la fuerza de la división de un grafo en módulos (comunidades).
    # Requiere encontrar las comunidades primero. Usamos el algoritmo de Louvain.
    try:
        # El algoritmo de Louvain busca la partición que maximiza la modularidad.
        # Usa el peso si está presente.
        comunidades = nx.community.louvain_communities(G_fuerza_proy, weight='weight')
        # La modularidad se calcula en base a la partición encontrada
        metricas["Modularidad"] = nx.community.modularity(G_fuerza_proy, comunidades, weight='weight')
    except Exception as e:
        # Esto puede fallar si el grafo es trivial o muy disperso.
        metricas["Modularidad"] = np.nan
        warnings.warn(f"No se pudo calcular la Modularidad: {e}")

    # 5. Formato de Salida
    # Asegurar que todos los valores sean flotantes o NaN para consistencia
    for k, v in metricas.items():
        if isinstance(v, (int, np.integer)):
            metricas[k] = float(v)
        elif isinstance(v, (float, np.floating)):
            pass # Ya es un float/numpy float
        elif v is None:
            metricas[k] = np.nan

    return metricas

In [14]:
calcular_metricas_tensor(multilayer_matrices_tensor, ids, "MULTI_GLOBAL", calcular_metricas_multigrafo)

Calculando métricas: 100%|██████████| 270/270 [19:32<00:00,  4.34s/it]

CSV guardado en: METRICAS_GRAFO/MULTI_GLOBAL.csv


# Calculos Multilayer (Local)

In [ ]:
calcular_metricas_tensor_local_por_nodo(multilayer_matrices_tensor, ids, "MULTI_LOCAL", calcular_metricas_grafo_local)

Procesando matrices: 100%|██████████| 270/270 [20:10<00:00,  4.48s/it]


CSV guardado en: METRICAS_GRAFO/fuerza_nodal_MULTI_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/grado_nodal_MULTI_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/eficiencia_local_MULTI_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/centralidad_intermediacion_MULTI_LOCAL.csv
CSV guardado en: METRICAS_GRAFO/centralidad_cercania_MULTI_LOCAL.csv
